In [16]:
import dimod
from dwave.system import DWaveSampler, EmbeddingComposite
from dwave.samplers import SimulatedAnnealingSampler 
from dimod.serialization.format import Formatter
import numpy as np
import random
import matplotlib.pyplot as plt

In [17]:
# solverPGS = PlanarGraphSolver()
samplerEC = EmbeddingComposite(DWaveSampler())
# samplerDWS = DWaveSampler()
# samplerSAS = SimulatedAnnealingSampler()  

In [18]:
def defCouplings(L):
    """
    Method for finding nearest neighbours couplings of an LxL lattice and randomly assigning the couplings a +1 or -1 value

    Outputs:
    [J_dict, J_matrix]
    J_dict - Formatted dictionary of couplings to be used in dwave's Sampler.sample_ising() functions
    J_matrix - Coupling matrix output to be used later for lowest energy state calculations using the findLowest(L,J) function
    """
    J_matrix = []
    J_dict = {}
    for i in range(L*L):
        row = np.zeros(shape = L*L, dtype = np.int8)
        
        if(i <L*(L-1)): # Add coupling to South neighbour
            try:
                row[i+L] = 1
            except IndexError:
                pass
        if(i%L!=(L-1)): # Add coupling to East neighbour, unless east-edge site
            row[i+1] = 1
        if(i%L==0): # Add coupling to "West" neighbour for west-edge sites
            row[i+L-1] = 1
        if(i<L):
            row[i+L*(L-1)] = 1
        # print(row)
        J_matrix.append(row)
    J_matrix = np.array(J_matrix)
    # By default only creates an upper triangular matrix, if working with QUBOs leave as is, if working in Ising model use a square matrix
    # J_matrix = J_matrix + J_matrix.T
    ij = np.where(J_matrix == 1)
    coupls = tuple(zip(*ij))

    for i in coupls:
        j_i = random.randrange(-1,2,2)
        J_dict.update({i:j_i})
        J_matrix[i[0]][i[1]] = j_i         
    
    return J_dict, J_matrix

In [93]:
def findLowest(L, J):
    """
    Find lowest energy of a spin configuration given some coupling matrix

    Arguments:
    L - Length and Width of 2-D square Ising model (i.e. 3x3 model, L=3)
    J - Coupling matrix

    Outputs: [minEnergy, minState, degenerateStates]
    int minEnergy - Energy of minimum energy state
    int array minState - Integer number representing state configuration of minimum energy state
    int array degenerateStates - Integer number(s) representing degenerate ground state configurations
    """
    # Add back J
    numSpins = L*L
    numStates = 2**numSpins
    minState = None
    minEnergy = 10**10
    degenerateStates = []
    
    for i in range(numStates):
        # s = np.zeros(numSpins, dtype=int)
        # # Convert number to state using binary conversion
        # for j in range(numSpins):
        #     if( (i & (1 << j)) != 0):
        #         s[j] = 1
        #     else:
        #         s[j] = -1
        s = numToState(i, numSpins)
                
        # Calculate energy of state 
        currEnergy = np.dot(s, np.dot(J,s))

        # Switch if necessary, raise degenerate ground state flag if applicable
        if(currEnergy < minEnergy):
            minEnergy = currEnergy
            # minState = s.copy()
            minState = i
            degenerateStates = [] # If a new lower state is found, formerly degenerate ground states just become degenerate states and are no longer of interest
        elif (abs(currEnergy - minEnergy) < 1e-10 ):
            degenerateStates.append(i)

    # degenerateStates = np.insert(degenerateStates, 0, minState, axis=0)

    return minEnergy, minState, degenerateStates

In [22]:
def numToState(i, numSpins):
    """
    Given a number and the number of states for a spin configuration convert the number to a binary number then a spin configuration with 1 -> +1 and 0 -> -1 spins

    Arguments:
    i - Number to convert
    numStates - Number of states to look at

    Output:
    s - Spin Configuration as an array with each spin a separate index
    """
    s = np.zeros(numSpins, dtype=int)

    for j in range(numSpins):
        if( (i & (1 << j)) != 0):
            s[j] = 1
        else:
            s[j] = -1

    return s

In [56]:
def dictToArr(statesDict):
    """
    Given a dictionary of site values, with the site index and site value, convert it to an array

    Purpose: DWave outputs site values in a dictionary, but the previously implemented methods output arrays

    Arguments:
    dict: Dictionary of sites to convert to array

    Output:
    arr - Array of site values
    """
    arr = []
    for i in statesDict:
        arr.append(statesDict[i])

    return np.array(arr)

In [87]:
def validation(stateArr, degenStates):
    L = len(stateArr)
    for i in range(len(degenStates)):
        print(stateArr)
        print(degenStates[i])
        print(L)
        temp = np.all(stateArr == numToState(degenStates[i],L*L))
        if(temp):
            print("Passed; index  " +str(i) + "of groundStates")

In [182]:
"""
What happens in this cell:
Defing matrix length, generate couplings, send job to Samplers, 
print out ground state energy sampled, print out ground state energy calculated, print validation result
"""

L = 5

# print(J)
nrgs = []

h = {}
J_coupl, J_matrix = defCouplings(L)
# print(len(J))
# offset = 0.0
# vartype = dimod.SPIN
# Currently the sampler.sample_ising samples an Ising model, if change to sampling a QUBO problem specifically use sampler.sample_qubo and remember to change the matrix formulation

for i in range(1):
    J_couplings, J_matrix = defCouplings(L)
    # minEnergy, minState, degenerateStates = findLowest(L,J_matrix)
    print("Finding lowest state...")
    minEnergy, minState, degenerateStates = findLowest(L,J_matrix)
    print("Submitted job to DWave Sampler")
    sampleset = samplerEC.sample_ising(h, J_couplings, num_reads = 1000)
    # nrgs.append(sampleset.first.energy)
    print("Sampled energy readings:")
    print("Sampled energy: " + str(sampleset.first.energy))
    print("Lowest energy possible: "+ str(minEnergy))
    # Add validation status print statement
    # sampleset.samples()[0]
    # print(sampleset.samples())

    print("\n--------------------------------------------------------------")
    print("State Validation")
    stateDict = sampleset.samples()[0]
    stateArr = dictToArr(stateDict)
    print(stateArr)
    minStateFnd = np.all(stateArr == numToState(minState, L*L))
    if(minStateFnd):
        # print(numToState(minState,L*L))
        print("Matched first min state")
    else:
        for i in range(len(degenerateStates)):
            temp = np.all(stateArr == numToState(degenerateStates[i],L*L))
            if(temp):
                print(numToState(degenerateStates[i],L*L))
                print("Degenerate State Index: " +str(i))

    print("\n--------------------------------------------------------------")
    print("Energy of all returned sampleset states:")
    for i in sampleset.samples():
        s = dictToArr(i)
        print(np.dot(s, np.dot(J_matrix,s)), end = ", ")

Finding lowest state...
Submitted job to DWave Sampler
Sampled energy readings:
Sampled energy: -32.0
Lowest energy possible: -32

--------------------------------------------------------------
State Validation
[-1 -1  1  1 -1 -1 -1  1 -1 -1  1 -1 -1 -1 -1  1 -1  1  1  1  1 -1 -1  1
 -1]
[-1 -1  1  1 -1 -1 -1  1 -1 -1  1 -1 -1 -1 -1  1 -1  1  1  1  1 -1 -1  1
 -1]
Degenerate State Index: 24

--------------------------------------------------------------
Energy of all returned sampleset states:
-32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -32, -28, -28, -28, -28, -28, -28, -28, -28, -28, -28, -28, -28, -28, -28, -24, 

In [ ]:
L = 6

# print(J)
nrgs = []

h = {}
J_coupl, J_matrix = defCouplings(L)
# print(len(J))
# offset = 0.0
# vartype = dimod.SPIN
# Currently the sampler.sample_ising samples an Ising model, if change to sampling a QUBO problem specifically use sampler.sample_qubo and remember to change the matrix formulation

for i in range(1):
    J_couplings, J_matrix = defCouplings(L)
    # minEnergy, minState, degenerateStates = findLowest(L,J_matrix)
    print("Finding lowest state...")
    minEnergy, minState, degenerateStates = findLowest(L,J_matrix)
    print("Submitted job to DWave Sampler")
    sampleset = samplerEC.sample_ising(h, J_couplings, num_reads = 1000)
    # nrgs.append(sampleset.first.energy)
    print("Sampled energy readings:")
    print("Sampled energy: " + str(sampleset.first.energy))
    print("Lowest energy possible: "+ str(minEnergy))
    # Add validation status print statement
    # sampleset.samples()[0]
    # print(sampleset.samples())

    print("\n--------------------------------------------------------------")
    print("State Validation")
    stateDict = sampleset.samples()[0]
    stateArr = dictToArr(stateDict)
    print(stateArr)
    minStateFnd = np.all(stateArr == numToState(minState, L*L))
    if(minStateFnd):
        # print(numToState(minState,L*L))
        print("Matched first min state")
    else:
        for i in range(len(degenerateStates)):
            temp = np.all(stateArr == numToState(degenerateStates[i],L*L))
            if(temp):
                print(numToState(degenerateStates[i],L*L))
                print("Degenerate State Index: " +str(i))

    print("\n--------------------------------------------------------------")
    print("Energy of all returned sampleset states:")
    for i in sampleset.samples():
        s = dictToArr(i)
        print(np.dot(s, np.dot(J_matrix,s)), end = ", ")

Finding lowest state...


In [ ]:
L = 10

# print(J)
nrgs = []

h = {}
J_coupl, J_matrix = defCouplings(L)
# print(len(J))
# offset = 0.0
# vartype = dimod.SPIN
# Currently the sampler.sample_ising samples an Ising model, if change to sampling a QUBO problem specifically use sampler.sample_qubo and remember to change the matrix formulation

for i in range(1):
    J_couplings, J_matrix = defCouplings(L)
    # minEnergy, minState, degenerateStates = findLowest(L,J_matrix)
    # print("Finding lowest state...")
    # minEnergy, minState, degenerateStates = findLowest(L,J_matrix)
    print("Submitted job to DWave Sampler")
    sampleset = samplerEC.sample_ising(h, J_couplings, num_reads = 100)
    # nrgs.append(sampleset.first.energy)
    print("Sampled energy readings:")
    print("Sampled energy: " + str(sampleset.first.energy))
    # print("Lowest energy possible: "+ str(minEnergy))
    # Add validation status print statement
    # sampleset.samples()[0]
    # print(sampleset.samples())

    # print("\n--------------------------------------------------------------")
    # print("State Validation")
    # stateDict = sampleset.samples()[0]
    # stateArr = dictToArr(stateDict)
    # print(stateArr)
    # minStateFnd = np.all(stateArr == numToState(minState, L*L))
    # if(minStateFnd):
    #     # print(numToState(minState,L*L))
    #     print("Matched first min state")
    # else:
    #     for i in range(len(degenerateStates)):
    #         temp = np.all(stateArr == numToState(degenerateStates[i],L*L))
    #         if(temp):
    #             print(numToState(degenerateStates[i],L*L))
    #             print("Degenerate State Index: " +str(i))

    print("\n--------------------------------------------------------------")
    print("Energy of all returned sampleset states:")
    for i in sampleset.samples():
        s = dictToArr(i)
        print(np.dot(s, np.dot(J_matrix,s)), end = ", ")

In [180]:
# Little blurb to manually check that 
stateInd = 4
print(dictToArr(sampleset.samples()[0]))
print(numToState(degenerateStates[stateInd],L*L))
print(np.all(dictToArr(sampleset.samples()[0]) == numToState(degenerateStates[4],L*L)))

[-1  1 -1  1 -1  1  1 -1 -1  1 -1 -1 -1  1  1  1]
[-1  1 -1  1 -1  1  1 -1 -1  1 -1 -1 -1  1  1  1]
True


Repeat process to 5x5 and 6x6 systems; Change spins to gaussian spins magnitude; Create function for comparing dwave states to state array; Check the result of sampleset.samples(); See how big a system dwave can accept/handle